In [ ]:
import os
import pandas as pd 
import datetime
import seaborn as sns

In [ ]:
#this notebook does the backtesting of strategy where we long the perpetual contracts
#and short delivery ccontracts in order to make profit
#this cell gets directory of given contract

long_type = "PERP"
symbol = ("BTCUSDT")
del_contract_date = "240329"

contract_date_numerical = datetime.datetime(int(del_contract_date[0:2])+2000, int(del_contract_date[2:4]), int(del_contract_date[4:6]))
contract_date_numerical = contract_date_numerical.timestamp() * 1000

# Get the list of all files and directories
if(long_type == "PERP"):
    path = "/Users/ahmetsalih/PycharmProjects/interest_calc/"+symbol
elif(long_type == "SPOT"):
    path = "/Users/ahmetsalih/PycharmProjects/interest_calc/"+symbol+"_SPOT"   
else:
    raise Exception("wrong long type")
dir_list = os.listdir(path)
print("Files and directories in '", path, "' :")
# prints all files
print(dir_list)


In [ ]:
# this cell categorizes and stores the file names

funding_rate_files = []
delivery_contract_files = []
holding_position_files = []

for file_name in dir_list:
    file_path = path + "/" + file_name
    if(file_name.split("-")[0] in [symbol, symbol+"_PERP"]):
        if(file_name.split("-")[1] == "fundingRate"):
            funding_rate_files.append(path + "/" + file_name)
        else:
            holding_position_files.append(path + "/" + file_name)
    else:
        del_name = file_name.split("-")[0]
        del_date = del_name.split("_")[1]
        print(del_name)
        if(del_name.split("_")[0] != symbol):continue
        if(del_date != del_contract_date):continue
        print("here")
        delivery_contract_files.append(path + "/" + file_name)
holding_position_files
    

In [ ]:
# this cell loads the data and aligns them

holding_position_data = pd.concat(map(pd.read_csv, holding_position_files))
holding_position_data = holding_position_data.sort_values(by=['open_time'], ignore_index=True)
holding_position_data = holding_position_data.sort_values(by=['open_time'], ignore_index=True)

delivery_data = pd.concat(map(pd.read_csv, delivery_contract_files))
delivery_data = delivery_data.sort_values(by=['open_time'], ignore_index=True)
delivery_data = delivery_data.drop_duplicates(subset=['open_time'], ignore_index=True)

lasttime = 0
for index, row in delivery_data.iterrows():
    if(row["open_time"] > contract_date_numerical):break
    lasttime = row["open_time"]
delivery_data = delivery_data[delivery_data["open_time"] <= lasttime]


open_time_min = min(delivery_data["open_time"])
close_time_max = max(delivery_data["close_time"])

open_time_min = max(open_time_min, min(holding_position_data["open_time"]))
close_time_max = min(close_time_max, max(holding_position_data["close_time"]))

if long_type == "PERP":
    funding_rate_data = pd.concat(map(pd.read_csv, funding_rate_files))
    funding_rate_data = funding_rate_data.sort_values(by=['calc_time'], ignore_index=True)
    funding_rate_data = funding_rate_data.drop_duplicates(subset=['calc_time'], ignore_index=True)
    open_time_min = max(open_time_min, min(funding_rate_data["calc_time"]))
    close_time_max = min(close_time_max, max(funding_rate_data["calc_time"]))

holding_position_data = holding_position_data[holding_position_data["open_time"] <= close_time_max]
holding_position_data = holding_position_data[holding_position_data["open_time"] >= open_time_min]

funding_rate_data = funding_rate_data[funding_rate_data["calc_time"] <= close_time_max]
funding_rate_data = funding_rate_data[funding_rate_data["calc_time"] >= open_time_min]

delivery_data = delivery_data[delivery_data['open_time'] <= close_time_max]
delivery_data = delivery_data[delivery_data["open_time"] >= open_time_min]

funding_rate_data

In [ ]:
#this cell calculates returns of this strategy and outputs it in a csv file

if(len(delivery_data) != len(holding_position_data)):
    print(len(delivery_data), len(holding_position_data))
    raise Exception("data sizes don't match")

curr_funding_rate = []
funding_rates_paid = []
times_till_maturity = []
funding_rate_index = len(funding_rate_data)-1
for i in range(len(delivery_data)-1, -1, -1):
    tick_open_time = delivery_data.iloc[i]["open_time"]
    tick_close_time = delivery_data.iloc[i]["close_time"]
    times_till_maturity.append(contract_date_numerical - tick_open_time)
    
    curr_funding_rate.append(funding_rate_data.iloc[funding_rate_index]["last_funding_rate"])
    
    paid_funding_rates_this_second = 0
    if(tick_open_time <= funding_rate_data.iloc[funding_rate_index]["calc_time"]
        and tick_close_time > funding_rate_data.iloc[funding_rate_index]["calc_time"]): 
        #and funding_rate_data.iloc[funding_rate_index]["calc_time"] <= tick_close_time):
        paid_funding_rates_this_second = funding_rate_data.iloc[funding_rate_index]["last_funding_rate"]
        funding_rate_index-=1
    new_fee = (holding_position_data.iloc[i]["open"]+holding_position_data.iloc[i]["close"])/2*paid_funding_rates_this_second
    if(funding_rates_paid == []):
        funding_rates_paid.append(new_fee)
    else:
        funding_rates_paid.append(funding_rates_paid[-1]+new_fee)
    
funding_rates_paid.reverse()
times_till_maturity.reverse()
curr_funding_rate.reverse()

data_fees = {"funding_rates_to_pay": funding_rates_paid, "times_till_maturity":times_till_maturity}
fee_df = pd.DataFrame.from_dict(data_fees)

leverage = 5
perp_parity_names = []
delivery_parity_names = []
date = []
time = []
perp_price = []
delivery_price = []
days_to_delivery = []
leverages = [] 
HPRs = []
APRs = []
Total_Funding_till_Maturity = []
Total_Funding_till_Maturity_annum = []
perc_diff = []


for i in range(len(delivery_data)):
    perp_parity_names.append(symbol)
    delivery_parity_names.append(symbol+"_"+del_contract_date)
    this_pt = datetime.datetime.fromtimestamp(delivery_data.iloc[i]["open_time"]/1000)
    
    date.append(this_pt.strftime("%m/%d/%Y"))
    time.append(this_pt.strftime("%H:%M"))
    
    is_open_prc = False
    if(delivery_data.iloc[i]["open"]/holding_position_data.iloc[i]["open"] > delivery_data.iloc[i]["close"]/holding_position_data.iloc[i]["close"]):
        is_open_prc = True
    
    if(is_open_prc):
        this_pt_perp_price = holding_position_data.iloc[i]["open"]
    else:
        this_pt_perp_price = holding_position_data.iloc[i]["close"]
    
    perp_price.append(this_pt_perp_price)
    
    if(is_open_prc):
        this_pt_delivery_price = delivery_data.iloc[i]["open"]
    else:
        this_pt_delivery_price = delivery_data.iloc[i]["close"]
    
    delivery_price.append(this_pt_delivery_price)
    
    this_days_to_delivery = times_till_maturity[i] / 86400000 #ms to days
    days_to_delivery.append(this_days_to_delivery)
    
    leverages.append(leverage)
    
    cost_as_funding = funding_rates_paid[i]
    HPR = leverage * (this_pt_delivery_price - this_pt_perp_price - cost_as_funding)/(this_pt_delivery_price + this_pt_perp_price) * 100
    APR = HPR * 365 / this_days_to_delivery
    
    HPRs.append(HPR)
    APRs.append(APR)
    
    Total_Funding_till_Maturity.append(100 * cost_as_funding/this_pt_perp_price)
    Total_Funding_till_Maturity_annum.append(365/this_days_to_delivery * 100 * cost_as_funding/this_pt_perp_price)

final_data_dict = {"perp parity name": perp_parity_names,
                   "delivery parity name": delivery_parity_names,
                   "date": date,
                   "time": time,
                   "perp price": perp_price,
                   "delivery price": delivery_price,
                   "days till maturity": days_to_delivery,
                   "leverage": leverages, 
                   "HPR":HPRs,
                   "APR":APRs,
                   "Total Funding Till Maturity": Total_Funding_till_Maturity,
                   "Total Funding Till Maturity Annualized": Total_Funding_till_Maturity_annum}
final_data_df = pd.DataFrame.from_dict(final_data_dict)

for i in range(len(final_data_df)):
    perc_diff.append(1000*((final_data_df.iloc[i]["delivery price"]/final_data_df.iloc[i]["perp price"])-1))
    
perc_diff_dict = {"perc_diff": perc_diff, "date":date}
perc_diff_df = pd.DataFrame.from_dict(perc_diff_dict)


final_data_df.to_csv(symbol+"_"+del_contract_date+".csv")

In [ ]:
#visualization
sns.set(rc={"figure.figsize":(35, 10)})

fin_data_len = len(final_data_df)
perc_diff_df_len = len(perc_diff_df)
final_data_df_clipped = final_data_df[:-fin_data_len//20]
perc_diff_clipped = perc_diff_df[:-perc_diff_df_len//20]


(sns.lineplot(data= final_data_df_clipped, x="date", y = "APR", color= "red")
 .set(title = f"minutley annual backtest results for {symbol}_{del_contract_date} between "
              f"{datetime.datetime.fromtimestamp(open_time_min//1000).date()} and {datetime.datetime.fromtimestamp(close_time_max//1000).date()}")) 
(sns.lineplot(data= final_data_df, x="date", y = "Total Funding Till Maturity Annualized", color= "blue"))
(sns.lineplot(data= perc_diff_clipped, x="date", y = "perc_diff", color = "orange"))


In [ ]:
#more visualization
funding_rates = []
dates = []
for i in range(len(funding_rate_data)):
    this_pt = datetime.datetime.fromtimestamp(funding_rate_data.iloc[i]["calc_time"]/1000)
    funding_rates.append(funding_rate_data.iloc[i]["last_funding_rate"])
    dates.append(this_pt.strftime("%m/%d/%Y"))
    
frdfd = {"funding_rates":funding_rates, "dates":dates}
frdf = pd.DataFrame.from_dict(frdfd)

ax1 =(sns.lineplot(data=final_data_df,x="date", y="perp price", color="blue"))
(sns.lineplot(data= final_data_df, x="date", y="delivery price", color="red"))
ax2 = ax1.twinx()

(sns.lineplot(data=frdf,x="dates", y="funding_rates", color="green", ax = ax2)
 .set(title = f"funding rate for {symbol}_{del_contract_date} between "
               f"{datetime.datetime.fromtimestamp(open_time_min//1000).date()} and {datetime.datetime.fromtimestamp(close_time_max//1000).date()}"))

In [ ]:
#more visualization
sns.set(rc={"figure.figsize":(35, 10)})

curr_funding_rate = curr_funding_rate[:-len(curr_funding_rate)//20]
perc_diff = perc_diff[:-len(perc_diff)//20]

rated_diff = []
for i in range(len(days_to_delivery)):
    rated_diff.append(perc_diff[i]/days_to_delivery[i])


fr_spread_dict = {"funding rate": curr_funding_rate, "rated diff":rated_diff, "percent difference":perc_diff, "days to delivery":days_to_delivery}
fr_spread_df = pd.DataFrame.from_dict(fr_spread_dict)
fr_spread_df = fr_spread_df[fr_spread_df["funding rate"] >= 0]
fr_spread_df = fr_spread_df[fr_spread_df["rated diff"] >= -0.5 ]



(sns.scatterplot(data=fr_spread_df,x="funding rate", y="rated diff", color="red")
 .set(title = f"funding rate for {symbol}_{del_contract_date} between "
               f"{datetime.datetime.fromtimestamp(open_time_min//1000).date()} and {datetime.datetime.fromtimestamp(close_time_max//1000).date()}"))
